# Frequency response demo (platform)

Set up notebook.

In [ ]:
# Import modules
import time
import numpy as np
from scipy import linalg
from scipy import signal
import sympy as sym
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import ae353_platform

# Enable interactive plots
%matplotlib widget

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

Create a simulator.

In [ ]:
simulator = ae353_platform.Simulator(
    display=False,
    roll=0.,
)

Define the state-space model.

In [ ]:
# Equilibrium point
q_e = 0.
v_e = 0.
tau_e = 0.

# Jacobian matrices
A = np.array([[0., 1.], [0., 0.]])
B = np.array([[0.], [1.]])

Do control design by eigenvalue placement.

In [ ]:
K = signal.place_poles(A, B, [-1. + 2. * 1j, -1. - 2. * 1j]).gain_matrix
print(f'K = np.array({K.tolist()})')

Do control implementation to track a sinusoidal desired platform angle.

In [ ]:
class Controller:
    def __init__(self, K, q_e, v_e, tau_e):
        self.K = K
        self.q_e = q_e
        self.v_e = v_e
        self.tau_e = tau_e
        
        # Tell the simlator to log the desired platform angle
        self.variables_to_log = ['q_des']
    
    def reset(self):
        self.q_des = 0.
    
    def run(
            self,
            t,
            platform_angle,
            platform_velocity,
            wheel_angle,
            wheel_velocity,
        ):
        
        # Find state
        x = np.array([
            platform_angle - self.q_e,
            platform_velocity - self.v_e,
        ])
        
        # Choose desired platform angle
        w = 1.
        self.q_des = np.sin(w * t)
        
        # Find desired state and input
        x_des = np.array([
            self.q_des - self.q_e,
            0. - self.v_e,
        ])
        u_des = np.array([
            0. - self.q_e,
        ])
        
        # Find input
        u = u_des - self.K @ (x - x_des)
        
        # Find torque applied to the platform
        tau = u[0] + self.tau_e
        
        # Find (and return) equal and opposite torque applied to the wheel
        wheel_torque = -tau
        return wheel_torque

Run simulation.

In [ ]:
# Create controller
controller = Controller(K, q_e, v_e, tau_e)
    
# Reset simulator
simulator.reset(
    platform_angle=0.,
    platform_velocity=0.,
    wheel_angle=0.,
    wheel_velocity=0.,
)

# Reset controller
controller.reset()

# Run simulator
data = simulator.run(
    controller,
    max_time=50.,
)

Plot results.

In [ ]:
t = data['t']
q = data['platform_angle']
v = data['platform_velocity']
tau = - data['wheel_torque']
q_des = data['q_des']

fig, (ax_q, ax_v, ax_tau) = plt.subplots(3, 1, figsize=(8, 8), sharex=True)

ax_q.plot(t, q, label=r'$q$', linewidth=4)
ax_q.plot(t, q_des, '--', label=r'$q$ (desired)', linewidth=3)
ax_q.grid()
ax_q.legend(fontsize=16)
ax_q.tick_params(labelsize=14)

ax_v.plot(t, v, label=r'$v$', linewidth=4)
ax_v.grid()
ax_v.legend(fontsize=16)
ax_v.tick_params(labelsize=14)

ax_tau.plot(t, tau, label=r'$\tau$', linewidth=4)
ax_tau.plot(t, np.ones_like(tau) * simulator.tau_max, '--', color='C1', linewidth=2)
ax_tau.plot(t, - np.ones_like(tau) * simulator.tau_max, '--', color='C1', linewidth=2)
ax_tau.grid()
ax_tau.legend(fontsize=16)
ax_tau.tick_params(labelsize=14)

ax_tau.set_xlabel('time (s)', fontsize=20)
ax_tau.set_xlim([data['t'][0], data['t'][-1]])

fig.tight_layout()
plt.show()

Define a matrix that picks the *only* element of $x_\text{des}$ that varies.

In [ ]:
M = np.array([[1., 0.]])

Define closed-loop model.

In [ ]:
Am = A - B @ K
Bm = - (A - B @ K) @ M.T
Cm = M

Define the function

$$H(s) = C_m (s - A_m)^{-1} B_m$$

where $s\in\mathbb{C}$ is a complex number.

In [ ]:
H = lambda s: (Cm @ linalg.inv(s * np.eye(Am.shape[0]) - Am) @ Bm).item()

Find the magnitude $|H(j\omega)|$ and the angle $\angle H(j\omega)$ of the complex number $H(j\omega)$ for different choices of frequency $\omega$ (radians / second).

Remember that any complex number $z\in\mathbb{C}$ can be written as

$$z = |z|e^{\angle z} = |z| \left( \cos(\angle z) + j \sin(\angle z) \right)$$

and that the period $T$ (in seconds) of a sinusoid with frequency $\omega$ (in radians / second) is

$$T = \dfrac{2\pi}{\omega}.$$

In [ ]:
omega = 1.
H_of_jomega = H(1j * omega)
magH = np.absolute(H_of_jomega)
angH = np.angle(H_of_jomega)

print(f'omega = {omega}')
print(f'H(j omega) = {H_of_jomega:.4f}')
print(f'mag = {magH:.4f}')
print(f'ang = {angH:.4f}')

Create Bode plot (i.e., plot magnitude and angle of $H(jw)$ as $w$ varies).

In [ ]:
omega = np.logspace(-1, 1, 250)
H_at_jomega = np.empty_like(omega, dtype=complex)
for i in range(len(omega)):
    H_at_jomega[i] = H(1j * omega[i])
mag = np.absolute(H_at_jomega)
ang = np.angle(H_at_jomega)

fig, (ax_mag, ax_ang) = plt.subplots(2, 1, figsize=(10, 5), sharex=True)
ax_mag.loglog(omega, mag, linewidth=3)
ax_mag.grid(visible=True, which='both', axis='both')
ax_mag.tick_params(labelsize=14)
ax_mag.yaxis.set_minor_locator(MultipleLocator(1e-1))
ax_mag.set_yticklabels([], minor=True)
ax_mag.set_ylabel('magnitude', fontsize=16)
ax_ang.semilogx(omega, ang, linewidth=2)
ax_ang.grid(visible=True, which='both', axis='both')
ax_ang.tick_params(labelsize=14)
ax_ang.yaxis.set_minor_locator(MultipleLocator(0.25))
ax_ang.set_yticklabels([], minor=True)
ax_ang.set_ylabel('angle (radians)', fontsize=16)
ax_ang.set_ylim([-np.pi, 0.])
ax_ang.set_xlim([min(omega), max(omega)])
ax_ang.set_xlabel(r'$\omega$ (radians / second)')
plt.show()